# 0. Imports

In [1]:
import sys
import os
import pandas as pd

current_dir = os.path.abspath(os.getcwd())
parent_dir = os.path.dirname(current_dir)
sys.path.append(os.path.join(parent_dir, 'scripts'))

import data_acquisition
import Dataframe
import feature_engineering
import Dataframe_milestone_2


# 1. Collecting Data

In [2]:
data_acquisition.collect_data()

The folder '../data/raw/2016_2017' already exists.
The folder '../data/raw/2017_2018' already exists.
The folder '../data/raw/2018_2019' already exists.
The folder '../data/raw/2019_2020' already exists.
The folder '../data/raw/2020_2021' already exists.


# 2. Cleaning Data

In [3]:
## environ 2 minutes

base_path = '../data/raw/'
df = Dataframe.load_all_seasons(base_path)
transformed_df = Dataframe.transformEventData(df)

Seasons:   0%|          | 0/5 [00:00<?, ?it/s]

Seasons: 100%|██████████| 5/5 [04:34<00:00, 54.90s/it]


## Ingénierie des caractéristiques 1

In [4]:
# Charger les données
data = pd.read_csv('../data/derivatives/dataframe.csv')
# Créer les nouvelles caractéristiques
feature_engineering.create_features1(data, '^201[6-9]02\d{4}$', 'features_train1.csv')

## Ingénierie des caractéristiques 2

In [5]:
## environ 7 minutes

base_path = '../data/raw/'
df = Dataframe_milestone_2.load_all_seasons(base_path)
transformed_df = Dataframe_milestone_2.transformEventData(df)

Seasons:   0%|          | 0/5 [00:00<?, ?it/s]

Seasons: 100%|██████████| 5/5 [06:34<00:00, 78.84s/it] 


In [7]:
Dataframe = pd.read_csv('../data/derivatives/dataframe_milestone_2.csv')
data = feature_engineering.create_features2(Dataframe, pattern=None)
data.to_csv('../data/derivatives/dataframe_milestone_2.csv', index=False)